In [1]:
import psycopg2
from datetime import datetime

In [2]:
def polygon_to_geotiff(polygon_coords):
    status = ''
    conn = psycopg2.connect(host="10.223.72.83" ,port="5433", database = "postgres" , user="postgres" , password="punpuntpasswd")
    try:
        polygon_wkt = "POLYGON((" + ",".join([f"{lon} {lat}" for lon, lat in polygon_coords]) + "))"
        
        query = f"""
        SELECT ST_AsTIFF(ST_UNION(
            ST_Clip(rast, ST_SetSRID(ST_MakeValid(ST_GeomFromText('{polygon_wkt}')), 4326))
        )) AS clipped_tiff
        FROM raster_results_1
        WHERE ST_Intersects(ST_SetSRID(ST_GeomFromText('{polygon_wkt}'), 4326), rast);
        """
        
        cursor = conn.cursor()
    
        cursor.execute("SET postgis.gdal_enabled_drivers = 'ENABLE_ALL';")
        cursor.execute(query)

        result = cursor.fetchone()
        cropped_tiff = result[0] if result else None
    
        out_image = cropped_tiff

        if cropped_tiff:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"output_{timestamp}.tiff"
            with open(f'/home/kea-trainee-punpun/Week2/polygon_tiff_output/{filename}', 'wb') as f:
                f.write(out_image.tobytes())
            status = f"File '{filename}' created."
        else:
            status = "No cropped raster data found."

        return status

    except Exception as e:
        print(f"Error: {e}")

    finally:
        cursor.close()
        conn.close()


In [3]:
polygon_coords = [
    # inside bound 2
    (102.26539189185458, 16.54356817627857), 
    (101.95606037474194, 16.08328194532515),
    (101.93394678768034, 16.647561388673765), 
    (102.36400115858939, 16.538869127991614),
    (102.26539189185458, 16.54356817627857)
    
    # inside bound 1
    # (101.86092, 15.57712), 
    # (101.87708, 15.57653),
    # (101.87435, 15.56847), 
    # (101.86335, 15.56638), 
    # (101.86092, 15.57712)

    # some are out
    # (101.3173, 15.4978),
    # (101.4995, 15.4964),
    # (101.5848, 15.3267),
    # (101.3166, 15.3301),
    # (101.3173, 15.4978)
    
    (101.30000000, 15.50764175),
    (101.58234912, 15.49990556), 
    (101.58062997, 15.30478166),
    (101.26774407, 15.44661181), 
    (101.30000000, 15.50764175)

    # all out
    # (101.7, 17.069),
    # (101.947, 17.065),
    # (101.828, 16.911),
    # (101.556, 16.877),
    # (101.662, 16.982),
    # (101.7, 17.069)

    # flip
    # (101.5, 15.5),
    # (100.0, 15.5),
    # (99.0, 16.0),
    # (101.5, 15.0),
    # (101.5, 15.5)
]

In [4]:
polygon_to_geotiff(polygon_coords)

"File 'output_20240708_052556.tiff' created."